In [1]:
#2020/08/13 Created by Kei Sanada
#
# 記事「ROE偏重経営、もろさ露呈　想定外に備えはあるか」
#https://r.nikkei.com/article/DGXMZO5742458030032020SHA000?disablepcview=&s=3
#「自己資本比率の高い企業の方が長期的には成長するのでは？」という記事の内容を元に
#自己資本比率の高い企業は、市場が不安定な時にも強いのではないかという仮説を確認してみる。
#2020/01/01-07/31でバックテストすると、確かに3月あたりはSPYよりも良いリターンだった。
#
#元ソースは　「Quantopian 日本語翻訳プロジェクト ドキュメント」
#「1. Getting Started」の「 1.10. リスクマネジメント」より。
#
#Algrthmsにはipynbファイルはインポートできないので、Notebooksにインポートして開いて、
#新しAlgrthmsにコピー＆ペーストしてください。
#
#改修例
#1.rebalanceのタイミングを変えてみる。
#2.自己資本比率以外の指標に変えてみる。

# Algorithm API インポート
import quantopian.algorithm as algo

# Optimize API インポート
import quantopian.optimize as opt

# Pipeline  インポート
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.psychsignal import stocktwits
from quantopian.pipeline.factors import SimpleMovingAverage

# built-in universe と Risk API method インポート
from quantopian.pipeline.filters import QTradableStocksUS
from quantopian.pipeline.experimental import risk_loading_pipeline

# モーニングスターのデータセットをインポート　Mod by Kei
from quantopian.pipeline.data.morningstar import Fundamentals

def initialize(context):
    # 制約パラメータ
    context.max_leverage = 1.0
    context.max_pos_size = 0.015
    context.max_turnover = 0.95

    # data pipelines を取り付ける
    algo.attach_pipeline(
        make_pipeline(),
        'data_pipe'
    )
    algo.attach_pipeline(
        risk_loading_pipeline(),
        'risk_pipe'
    )

    # rebalance 関数をスケジュール
    # Kei rebalanceのタイミングを変えると結果が変わるかも？
    algo.schedule_function(
        rebalance,
        algo.date_rules.week_start(),
        algo.time_rules.market_open(),
    )


def before_trading_start(context, data):
    # pipeline出力を取得し、contextに格納する。
    context.pipeline_data = algo.pipeline_output('data_pipe')

    context.risk_factor_betas = algo.pipeline_output('risk_pipe')

#ADD by Kei　
#モーニングスターのデータセットのtotal_equityとtotal_assetsを使って自己資本比率を計算
#ユニバースはQTradableStocksUS()を使用。
#QTradableStocksUS()かつ、自己資本比率の計算結果がNULLでなく、有効な値であることをスクリーンに使う。

def create_factor():
    qtu = QTradableStocksUS()
    
    total_equity = Fundamentals.total_equity.latest
    total_assets = Fundamentals.total_assets.latest
    capital_adequacy_ratio = total_equity / total_assets  * 100 #自己資本比率の計算
    alpha_factor = capital_adequacy_ratio
    screen = qtu & ~alpha_factor.isnull() & alpha_factor.isfinite()
    return alpha_factor, screen
    
# Pipeline definition
def make_pipeline():

    #Add by Kei
    alpha_factor, screen = create_factor()

    return Pipeline(
        columns={
            'alpha_factor': alpha_factor, #Mod by Kei
        },
        screen=screen
    )


def rebalance(context, data):
    # Mod by Kei pipeline 出力から alpha_factor を取り出す
    alpha = context.pipeline_data.alpha_factor

    if not alpha.empty:
        # MaximizeAlpha objective 作成
        objective = opt.MaximizeAlpha(alpha)

        # ポジションサイズ制約
        constrain_pos_size = opt.PositionConcentration.with_equal_bounds(
            -context.max_pos_size,
            context.max_pos_size
        )

        # ターゲットポートフォリオレバレッジ制約
        max_leverage = opt.MaxGrossExposure(context.max_leverage)

        # ロング（買い持ち）とショート（売り持ち）のサイズをだいたい同じに合わせる
        dollar_neutral = opt.DollarNeutral()

        # ポートフォリオの出来高の制約
        max_turnover = opt.MaxTurnover(context.max_turnover)

        # ターゲットポートフォリオのリスクエクスポージャーを制限する。
        # デフォルト値は、セクターエクスポージャーの最大値は0.2
        # スタイルエクスポージャーの最大値は0.4
        factor_risk_constraints = opt.experimental.RiskModelExposure(
            context.risk_factor_betas,
            version=opt.Newest
        )

        # 目的関数と制約リストを使ってポートフォリオをリバランスする
        algo.order_optimal_portfolio(
            objective=objective,
            constraints=[
                constrain_pos_size,
                max_leverage,
                dollar_neutral,
                max_turnover,
                factor_risk_constraints,
            ]
        )

ImportError: No module named 'quantopian.algorithm'